In [1]:
import numpy as np
import cv2
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
def convert_to_yolo_format(bbox, image_width, image_height):
    x, y, width, height = bbox
    center_x = x + width / 2
    center_y = y + height / 2
    return [center_x, center_y, width, height]

In [3]:
import pandas as pd

In [278]:
data = pd.read_csv(r"\\vector.e3b.columbia.edu\e3b\Globus Database\UWIN Tagged Photos\batch_014\detections_to_update_2024-02-22.csv")

In [279]:
parent_dir = r"D:\UWIN Tagged Photos Outputs"
batch_results_dir = r"D:\UWIN Tagged Photos Outputs\Batchwise Results"

In [280]:
json_filenames = os.listdir(batch_results_dir)

In [281]:
import json

In [282]:
json_file = os.path.join(batch_results_dir, json_filenames[13])
json_file

'D:\\UWIN Tagged Photos Outputs\\Batchwise Results\\results_batch_014.json'

In [283]:
with open(json_file, 'r') as file:
    results = json.load(file)

In [284]:
results[11]

{'file': '//vector.e3b.columbia.edu/e3b/Globus Database/UWIN Tagged Photos/batch_014/VID29215-00011.jpg',
 'max_detection_conf': 0.966,
 'detections': [{'category': '2',
   'conf': 0.966,
   'bbox': [0.09923, 0, 0.7345, 0.979]}]}

In [285]:
from tqdm import tqdm
data_dict = {}
for r in tqdm(results):
    dets = r['detections']
    if len(dets)==1 and dets[0]['conf']>0.2:
        fname = os.path.basename(r['file'])
        bbox = dets[0]['bbox']
        x,y,h,w = bbox
        data_dict[fname] = {'fname':fname, 'x':x, 'y':y, 'h':h, 'w':w}
print('Done')

100%|██████████████████████████████████████████████████████████████████████████| 2306/2306 [00:00<00:00, 513488.27it/s]

Done


In [286]:
len(data_dict)

1024

In [287]:
non_empty = list(data_dict.keys())

In [288]:
filtered_data = data[data['photoName'].isin(non_empty)]

In [289]:
filtered_data.shape

(1083, 14)

In [290]:
filtered_data.drop('Unnamed: 0', axis=1, inplace=True)

In [291]:
filtered_data

,photoName,timestamp,speciesID,species,count,siteName,siteAbbrev,longitude,latitude,validationStatus,group,updateCommonName,updateNumIndividuals
1,VID34099-00037.jpg,2022-12-28 00:46:42,4,White-tailed deer,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
4,VID34099-00040.jpg,2022-12-28 17:00:05,4,White-tailed deer,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
5,VID34099-00041.jpg,2022-12-28 17:00:06,4,White-tailed deer,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
6,VID34099-00042.jpg,2022-12-28 17:00:07,4,White-tailed deer,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
8,VID34099-00044.jpg,2022-12-28 17:06:09,4,White-tailed deer,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,VID32814-00878.jpg,2022-11-02 14:11:51,2,Eastern gray squirrel,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
2434,VID32814-00884.jpg,2022-11-02 21:55:39,2,Eastern gray squirrel,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
2435,VID32814-00885.jpg,2022-11-02 21:55:40,2,Eastern gray squirrel,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
2442,VID32814-00892.jpg,2022-11-03 18:27:06,2,Eastern gray squirrel,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN


In [292]:
duplicate_values = filtered_data['photoName'][filtered_data['photoName'].duplicated(keep=False)]

# Filtering rows with values in C1 that occur more than once
filtered_df_no_dup = filtered_data[~filtered_data['photoName'].isin(duplicate_values)]

In [293]:
filtered_df_no_dup

,photoName,timestamp,speciesID,species,count,siteName,siteAbbrev,longitude,latitude,validationStatus,group,updateCommonName,updateNumIndividuals
1,VID34099-00037.jpg,2022-12-28 00:46:42,4,White-tailed deer,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
4,VID34099-00040.jpg,2022-12-28 17:00:05,4,White-tailed deer,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
5,VID34099-00041.jpg,2022-12-28 17:00:06,4,White-tailed deer,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
6,VID34099-00042.jpg,2022-12-28 17:00:07,4,White-tailed deer,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
8,VID34099-00044.jpg,2022-12-28 17:06:09,4,White-tailed deer,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,VID32814-00878.jpg,2022-11-02 14:11:51,2,Eastern gray squirrel,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
2434,VID32814-00884.jpg,2022-11-02 21:55:39,2,Eastern gray squirrel,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
2435,VID32814-00885.jpg,2022-11-02 21:55:40,2,Eastern gray squirrel,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN
2442,VID32814-00892.jpg,2022-11-03 18:27:06,2,Eastern gray squirrel,1,Westbury Equestrian Center,WbEc,-73.583809,40.78421,1,14,NaN,NaN


In [294]:
new_data = filtered_df_no_dup[['photoName', 'speciesID', 'species']]

In [295]:
new_data

,photoName,speciesID,species
1,VID34099-00037.jpg,4,White-tailed deer
4,VID34099-00040.jpg,4,White-tailed deer
5,VID34099-00041.jpg,4,White-tailed deer
6,VID34099-00042.jpg,4,White-tailed deer
8,VID34099-00044.jpg,4,White-tailed deer
...,...,...,...
2428,VID32814-00878.jpg,2,Eastern gray squirrel
2434,VID32814-00884.jpg,2,Eastern gray squirrel
2435,VID32814-00885.jpg,2,Eastern gray squirrel
2442,VID32814-00892.jpg,2,Eastern gray squirrel


In [296]:
new_data['x'] = new_data['photoName'].apply(lambda name: data_dict[name]['x'])
new_data['y'] = new_data['photoName'].apply(lambda name: data_dict[name]['y'])
new_data['w'] = new_data['photoName'].apply(lambda name: data_dict[name]['w'])
new_data['h'] = new_data['photoName'].apply(lambda name: data_dict[name]['h'])

In [297]:
new_data

,photoName,speciesID,species,x,y,w,h
1,VID34099-00037.jpg,4,White-tailed deer,0.2880,0.5298,0.16340,0.13970
4,VID34099-00040.jpg,4,White-tailed deer,0.3390,0.5112,0.16600,0.13430
5,VID34099-00041.jpg,4,White-tailed deer,0.3607,0.5041,0.17220,0.11280
6,VID34099-00042.jpg,4,White-tailed deer,0.3603,0.5121,0.16450,0.11170
8,VID34099-00044.jpg,4,White-tailed deer,0.1976,0.5112,0.17580,0.14570
...,...,...,...,...,...,...,...
2428,VID32814-00878.jpg,2,Eastern gray squirrel,0.6555,0.4580,0.04994,0.03873
2434,VID32814-00884.jpg,2,Eastern gray squirrel,0.7204,0.4530,0.05437,0.07081
2435,VID32814-00885.jpg,2,Eastern gray squirrel,0.7205,0.4539,0.05348,0.07014
2442,VID32814-00892.jpg,2,Eastern gray squirrel,0.6826,0.4435,0.05082,0.06831


In [298]:
new_data.to_csv(r"D:\UWIN Tagged Photos Outputs\single box images\batch_014.csv", index=False)